In [1]:
from datasets import load_dataset

ds = load_dataset("ai4bharat/samanantar", "hi",streaming=True)

/home/sarthak/Desktop/VakyaShuddhi/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from indicnlp.normalize.indic_normalize import DevanagariNormalizer
from indicnlp.tokenize import indic_tokenize

class hindi_pipeline():
  def __init__(self):

    self.normalizer = DevanagariNormalizer(
        lang='hi',
        remove_nuktas=True,
        nasals_mode='to_anusvaara_strict',
        do_normalize_chandras=False,
        do_normalize_vowel_ending=False
    )

    self.tokenizer = indic_tokenize

    return


  def normalize(self,text):
    clean_text = self.normalizer.normalize(text)
    return clean_text

  def tokenize(self,text):
    tokens = self.tokenizer.trivial_tokenize(text,'hi')
    return tokens

In [ ]:
from collections import Counter
import json

pipe = hindi_pipeline()
chunks = 100000
chunk_tokens = []
word_counter = Counter()

for i, row in enumerate(ds['train'], 1): 
    clean_text = pipe.normalize(row['tgt'])
    tokens = pipe.tokenize(clean_text)
    chunk_tokens.extend(tokens)

    if i % chunks == 0:
        word_counter.update(chunk_tokens)
        chunk_tokens = []
        print(f"Processed {i} sentences")

if chunk_tokens:
    word_counter.update(chunk_tokens)

with open('word_counts.json','w') as f:
    json.dump(word_counter,f,ensure_ascii=False,indent=4)